In [57]:
import numpy as np  # Importamos NumPy para trabajar con matrices
import random  # Importamos random para la generación de números aleatorios
from tqdm import tqdm  # Importamos tqdm para mostrar barras de progreso en bucles

class Board():
    def __init__(self):
        self.state = np.zeros((6,7))  # Creamos una matriz de 6x7 llena de ceros para representar el tablero del juego 4 en raya

    def valid_moves(self):
        # Devuelve una lista de tuplas que representan las coordenadas de las casillas vacías en el tablero
        return [(i, j) for i in range(6) for j in range(7) if self.state[i, j] == 0]

    def update(self, symbol, row, col):
        if self.state[row, col] == 0:  # Verifica si la casilla está vacía
            self.state[row, col] = symbol  # Coloca el símbolo del jugador en la casilla especificada
            return row, col  # Devuelve las coordenadas de la casilla actualizada
        else:
            raise ValueError("Movimiento ilegal !")  # Si la casilla no está vacía, se lanza un error

    def is_game_over(self):
        # Comprobamos si alguna fila, columna o diagonal tiene 4 fichas del mismo jugador
        for i in range(6):
            for j in range(4):
                if sum(self.state[i, j:j+4]) == 4 or sum(self.state[j:j+4, i]) == 4:
                    return 1  # Devuelve 1 si hay un ganador (jugador 1)
                elif sum(self.state[i, j:j+4]) == -4 or sum(self.state[j:j+4, i]) == -4:
                    return -1  # Devuelve -1 si hay un ganador (jugador 2)
        for i in range(3):
            for j in range(4):
                if sum([self.state[i+k, j+k] for k in range(4)]) == 4 or sum([self.state[i+k, j+k] for k in range(4)]) == -4:
                    return 1
                elif sum([self.state[i+k, j+k] for k in range(4)]) == -4 or sum([self.state[i+k, j+k] for k in range(4)]) == -4:
                    return -1
        for i in range(3):
            for j in range(4):
                if sum([self.state[i+k, 6-j-k] for k in range(4)]) == 4 or sum([self.state[i+k, 6-j-k] for k in range(4)]) == -4:
                    return 1
                elif sum([self.state[i+k, 6-j-k] for k in range(4)]) == -4 or sum([self.state[i+k, 6-j-k] for k in range(4)]) == -4:
                    return -1
        # Si no hay ganador y no hay movimientos válidos restantes, devuelve 0 para indicar un empate
        if len(self.valid_moves()) == 0:
            return 0
        # Si el juego aún no ha terminado, devuelve None
        return None

    def reset(self):
        self.state = np.zeros((6,7))  # Resetea el tablero, llenándolo nuevamente con ceros


In [52]:
class Agent():
    def __init__(self, alpha=0.1, epsilon=0.1):
        self.q_values = {}  # Inicializa un diccionario para almacenar los valores Q, donde las claves son pares (estado, acción)
        self.alpha = alpha  # Tasa de aprendizaje (alfa)
        self.epsilon = epsilon  # Probabilidad de exploración (epsilon)

    def choose_action(self, board):
        if np.random.rand() < self.epsilon:
            return random.choice(board.valid_moves())  # Selección aleatoria de una acción válida
        else:
            state = str(board.state)  # Convierte el estado actual del tablero a una cadena
            actions = board.valid_moves()  # Obtiene las acciones válidas disponibles en el tablero
            values = [self.q_values.get((state, action), 0) for action in actions]  # Obtiene los valores Q para cada acción
            return actions[np.argmax(values)]  # Elige la acción con el valor Q máximo

    def update_q_values(self, state, action, reward, next_state):
        # Actualiza los valores Q utilizando la regla de aprendizaje de temporal difference (TD)
        current_q_value = self.q_values.get((state, action), 0)  # Obtiene el valor Q actual para el par (estado, acción)
        next_max_q_value = max([self.q_values.get((next_state, next_action), 0) for next_action in Board().valid_moves()])  # Obtiene el máximo valor Q para el siguiente estado
        new_q_value = current_q_value + self.alpha * (reward + next_max_q_value - current_q_value)  # Calcula el nuevo valor Q
        self.q_values[(state, action)] = new_q_value  # Actualiza el valor Q para el par (estado, acción)


In [53]:
# Creamos el tablero y los agentes
board = Board()
agent1 = Agent(alpha=0.1, epsilon=0.1)
agent2 = Agent(alpha=0.1, epsilon=0.1)

In [54]:
# Entrenamos los agentes jugando entre ellos
for _ in tqdm(range(10000)):  # Realizamos 10,000 iteraciones del bucle
    board.reset()  # Reseteamos el tablero al estado inicial
    state = str(board.state)  # Obtenemos el estado actual del tablero como una cadena
    while True:  # Comenzamos un bucle hasta que el juego termine
        # Agente 1 elige una acción
        action = agent1.choose_action(board)  # Elige una acción basada en la política epsilon-greedy
        row, col = board.update(1, action[0], action[1])  # Actualiza el tablero con la acción elegida
        next_state = str(board.state)  # Obtiene el próximo estado del tablero como una cadena
        reward = board.is_game_over()  # Comprueba si el juego ha terminado y devuelve la recompensa
        if reward is not None:  # Si el juego ha terminado
            # Actualiza los valores Q de ambos agentes con la recompensa recibida y el próximo estado
            agent1.update_q_values(state, action, reward, next_state)
            agent2.update_q_values(state, action, reward, next_state)
            break  # Sale del bucle
        else:  # Si el juego no ha terminado
            # Actualiza los valores Q de ambos agentes con una recompensa de 0 para el próximo estado
            agent1.update_q_values(state, action, 0, next_state)
            agent2.update_q_values(state, action, 0, next_state)
        state = next_state  # Actualiza el estado actual del tablero al próximo estado

        # Agente 2 elige una acción
        action = agent2.choose_action(board)  # Elige una acción basada en la política epsilon-greedy
        row, col = board.update(-1, action[0], action[1])  # Actualiza el tablero con la acción elegida
        next_state = str(board.state)  # Obtiene el próximo estado del tablero como una cadena
        reward = board.is_game_over()  # Comprueba si el juego ha terminado y devuelve la recompensa
        if reward is not None:  # Si el juego ha terminado
            # Actualiza los valores Q de ambos agentes con la recompensa recibida y el próximo estado
            agent1.update_q_values(state, action, reward, next_state)
            agent2.update_q_values(state, action, reward, next_state)
            break  # Sale del bucle
        else:  # Si el juego no ha terminado
            # Actualiza los valores Q de ambos agentes con una recompensa de 0 para el próximo estado
            agent1.update_q_values(state, action, 0, next_state)
            agent2.update_q_values(state, action, 0, next_state)
        state = next_state  # Actualiza el estado actual del tablero al próximo estado


100%|██████████| 10000/10000 [00:31<00:00, 320.44it/s]


In [56]:
# Evaluamos los agentes en 10 juegos
wins = [0, 0, 0]  # Número de victorias de cada agente y empates
for _ in tqdm(range(10)):  # Iteramos sobre 10 juegos
    board.reset()  # Reseteamos el tablero al estado inicial
    state = str(board.state)  # Obtenemos el estado actual del tablero como una cadena
    while True:  # Comenzamos un bucle hasta que el juego termine
        # Agente 1 elige una acción
        action = agent1.choose_action(board)  # Elige una acción basada en la política epsilon-greedy
        row, col = board.update(1, action[0], action[1])  # Actualiza el tablero con la acción elegida
        reward = board.is_game_over()  # Comprueba si el juego ha terminado y devuelve la recompensa
        if reward is not None:  # Si el juego ha terminado
            if reward == 1:  # Si el agente 1 ha ganado
                wins[0] += 1  # Incrementa el contador de victorias del agente 1
            elif reward == -1:  # Si el agente 2 ha ganado
                wins[1] += 1  # Incrementa el contador de victorias del agente 2
            else:  # Si hay un empate
                wins[2] += 1  # Incrementa el contador de empates
            break  # Sale del bucle
        state = str(board.state)  # Actualiza el estado actual del tablero como una cadena

        # Agente 2 elige una acción
        action = agent2.choose_action(board)  # Elige una acción basada en la política epsilon-greedy
        row, col = board.update(-1, action[0], action[1])  # Actualiza el tablero con la acción elegida
        reward = board.is_game_over()  # Comprueba si el juego ha terminado y devuelve la recompensa
        if reward is not None:  # Si el juego ha terminado
            if reward == 1:  # Si el agente 1 ha ganado
                wins[0] += 1  # Incrementa el contador de victorias del agente 1
            elif reward == -1:  # Si el agente 2 ha ganado
                wins[1] += 1  # Incrementa el contador de victorias del agente 2
            else:  # Si hay un empate
                wins[2] += 1  # Incrementa el contador de empates
            break  # Sale del bucle
        state = str(board.state)  # Actualiza el estado actual del tablero como una cadena

# Imprimimos los resultados
print("Victorias del agente 1:", wins[0])
print("Victorias del agente 2:", wins[1])
print("Empates:", wins[2])


100%|██████████| 10/10 [00:00<00:00, 198.94it/s]

Victorias del agente 1: 9
Victorias del agente 2: 1
Empates: 0
